In [2]:
from langchain_community.document_loaders import WikipediaLoader

In [3]:
docs = WikipediaLoader(query="brazil", load_max_docs=10).load()

d:\Python\Python310\envd\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\Python\Python310\envd\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [4]:
docs

[Document(page_content='Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw] ), officially the Federative Republic of Brazil (Portuguese: ), is the largest country in South America and in Latin America. Brazil is the world\'s fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the only country in the Americas to have Portuguese as an official language. It is one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world, and the most populous Roman Catholic-majority country.\nBounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi). It borders all other countries and territories in South America except Ecuador and Chile and covers roughly half of the continent\'s land area. Its Amazon basin includes a vast tropical f

In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chat_models import ChatLiteLLM
from dotenv import load_dotenv
load_dotenv()

model = ChatLiteLLM(model="huggingface/codellama/CodeLlama-34b-Instruct-hf")

In [6]:
hp = HumanMessagePromptTemplate.from_template("Summarize the document in one sentence:\n {doc}")
prompt = ChatPromptTemplate.from_messages([hp])
result = model(prompt.format_prompt(doc=docs[0].page_content).to_messages())

d:\Python\Python310\envd\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [7]:
result

AIMessage(content="Brazil, officially the Federative Republic of Brazil, is a country in South America and in Latin America. Brazil is the world's largest country, has the most populous city, and is the largest producer of coffee in the world; it is one of the world's largest economies and is an upper-middle class emerging power that is an emerging power and a major non-NATO ally of the United States.")

In [8]:
docs[0].page_content

'Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw] ), officially the Federative Republic of Brazil (Portuguese: ), is the largest country in South America and in Latin America. Brazil is the world\'s fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the only country in the Americas to have Portuguese as an official language. It is one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world, and the most populous Roman Catholic-majority country.\nBounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi). It borders all other countries and territories in South America except Ecuador and Chile and covers roughly half of the continent\'s land area. Its Amazon basin includes a vast tropical forest, home to diverse 

In [9]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 100, chunk_overlap = 0
)
texts = text_splitter.split_text(docs[0].page_content)

Created a chunk of size 702, which is longer than the specified 100


In [10]:
texts

["Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw] ), officially the Federative Republic of Brazil (Portuguese: ), is the largest country in South America and in Latin America. Brazil is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the only country in the Americas to have Portuguese as an official language. It is one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world, and the most populous Roman Catholic-majority country.\nBounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi). It borders all other countries and territories in South America except Ecuador and Chile and covers roughly half of the continent's land area. Its Amazon basin includes a vast tropical forest, home to diverse w

In [11]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [12]:
embeddings_model = HuggingFaceInstructEmbeddings(model_name = "hkunlp/instructor-xl")

d:\Python\Python310\envd\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [13]:
embeddings = embeddings_model.embed_documents(texts)

In [14]:
from langchain.vectorstores import FAISS

In [15]:
vectorstore = FAISS.from_texts(texts=texts, embedding=embeddings_model)

In [16]:
vectorstore.similarity_search("Where the word Brazil comes from?")

[Document(page_content='The word "Brazil" likely comes from the Portuguese word for brazilwood, a tree that once grew plentifully along the Brazilian coast. In Portuguese, brazilwood is called pau-brasil, with the word brasil commonly given the etymology "red like an ember", formed from brasa ("ember") and the suffix -il (from -iculum or -ilium). As brazilwood produces a deep red dye, it was highly valued by the European textile industr'),
 Document(page_content="Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw] ), officially the Federative Republic of Brazil (Portuguese: ), is the largest country in South America and in Latin America. Brazil is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the only country in the Americas to have Portuguese as an official language. It is one of the most multicult

In [17]:
retriever = vectorstore.as_retriever()

In [18]:
sk = {"score_threshold": 0.9, "k":1}

In [19]:
result_docs = retriever.get_relevant_documents("Which are the brazilian most populous city?",search_kwargs = sk)

In [20]:
def print_docs(docs):
    print(f"\n{'-' * 200}\n".join([f"Documento - {i+1}:\n" + j.page_content for i, j in enumerate(docs)]))

In [21]:
print_docs(result_docs)

Documento - 1:
Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw] ), officially the Federative Republic of Brazil (Portuguese: ), is the largest country in South America and in Latin America. Brazil is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the only country in the Americas to have Portuguese as an official language. It is one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world, and the most populous Roman Catholic-majority country.
Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi). It borders all other countries and territories in South America except Ecuador and Chile and covers roughly half of the continent's land area. Its Amazon basin includes a vast tropical forest, home 

In [22]:
from langchain.llms import HuggingFaceHub
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

llm = HuggingFaceHub(repo_id = "google/flan-t5-xxl", model_kwargs = {"temperature": 0.5, "max_length": 512})

compressor = LLMChainExtractor.from_llm(llm)
compressor_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

docs_comp = compressor_retriever.get_relevant_documents("Which are the brazilian most populous city?")
print_docs(docs_comp)

d:\Python\Python310\envd\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
d:\Python\Python310\envd\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
d:\Python\Python310\envd\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Documento - 1:
Its capital is Braslia, and its most populous city is So Paulo.


In [30]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

In [51]:
str = docs_comp[0].page_content

In [40]:
#memory.save_context({"input":"brazil"}, {"output": str})

In [59]:
import pickle 
new_memory_load = open('memory.pkl', 'rb').read()
from langchain.chains import ConversationChain

In [64]:
memory = pickle.loads(new_memory_load)
memory.save_context({"input":"brazil"}, {"output": str})
reload_conversation = ConversationChain(llm = llm, memory = memory)

In [65]:
reload_conversation.memory.chat_memory.messages

[HumanMessage(content='HI!'),
 AIMessage(content="Hello! I'm happy to chat with you. Is there something specific you'd like to talk about?"),
 HumanMessage(content='Tell me about you'),
 AIMessage(content="Sure, I'd be happy to tell you about myself! I'm just an AI, which stands for artificial intelligence. I'm a computer program that's been designed to process and generate human-like language. I'm capable of understanding and responding to language inputs, and can even generate my own responses.\n\nI was trained on a large dataset of text from the internet, which allows me to have a wide range of knowledge and be able to answer many different types of questions. I'm constantly learning and improving my abilities, so I can become more helpful and accurate over time.\n\nAs for my personality, I don't have personal preferences or emotions like humans do. However, I am designed to be helpful and assist users with their questions and tasks. I'm here to provide information and support, so p

In [66]:
reload_conversation.predict(input="how are you")

"I'm happy to chat with you. Is there something specific you'd like to talk about?"

In [67]:
reload_conversation.memory.save_context

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='HI!'), AIMessage(content="Hello! I'm happy to chat with you. Is there something specific you'd like to talk about?"), HumanMessage(content='Tell me about you'), AIMessage(content="Sure, I'd be happy to tell you about myself! I'm just an AI, which stands for artificial intelligence. I'm a computer program that's been designed to process and generate human-like language. I'm capable of understanding and responding to language inputs, and can even generate my own responses.\n\nI was trained on a large dataset of text from the internet, which allows me to have a wide range of knowledge and be able to answer many different types of questions. I'm constantly learning and improving my abilities, so I can become more helpful and accurate over time.\n\nAs for my personality, I don't have personal preferences or emotions like humans do. However, I am designed to be helpful and assist users with their question